<a href="https://colab.research.google.com/github/Autobot37/jupyter/blob/main/CustomHuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate
!pip install transformers

In [ ]:
from transformers import Trainer

In [ ]:
import transformers
import numpy as np

In [ ]:
import torch
from torch import nn

In [ ]:
class dataset(torch.utils.data.Dataset):
  def __init__(self):
    super().__init__()
    self.nums = torch.arange(1000)
  
  def __len__(self):
    return 999
  
  def __getitem__(self,idx):
    return {"given":torch.tensor(self.nums[idx]).unsqueeze(0),"labels":torch.tensor(torch.square(self.nums[idx]))}

In [ ]:
ds = dataset()

In [ ]:
TFout = transformers.modeling_outputs.BaseModelOutput

In [ ]:
from torch.nn import functional as F

In [ ]:
l1 = nn.Linear(2,1)
l1.weight.dtype

torch.float32

In [ ]:
class model(nn.Module):
  def __init__(self):
    super().__init__()
    self.l1 = nn.Linear(1,8)
    self.l2 =  nn.Linear(8,1)
    
  
  def forward(self,x):
    x = x.to(self.l1.weight.dtype)  # Convert input tensor to the same data type as the weight matrix
    out = F.relu(self.l2(F.relu(self.l1(x))))
    return out


In [ ]:
class CTrainer(Trainer):
  def get_train_dataloader(self):
    return torch.utils.data.DataLoader(ds,batch_size=4,shuffle=False)
  
  def get_eval_dataloader(self):
    return torch.utils.data.DataLoader(ds,batch_size=4,shuffle=True)
  
  def compute_loss(self,model,inputs,returns_output=False):
    y_hat = model(inputs.get("given"))
    y = inputs.get("labels")
    loss = torch.mean((y-y_hat)**2)

    return loss
  
  
  




    




In [ ]:
from transformers import TrainingArguments

In [ ]:
args = TrainingArguments(output_dir="test",num_train_epochs=1)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
model = model()

In [ ]:
trainer = CTrainer(
    model=model,
    args=args,
    train_dataset=ds,
    eval_dataset=ds,
)

In [ ]:
trainer.train()

In [ ]:
import torchvision.datasets as datasets

In [ ]:
class dataset(torch.utils.data.Dataset):
  def __init__(self):
    self.train_dataset = datasets.MNIST(root='contents', train=True, download=True)
    self.train_dataset.data = self.train_dataset.data[:1000]
    self.train_dataset.targets = self.train_dataset.targets[:1000]  

  def __getitem__(self, idx):
    img = self.train_dataset.data[idx]
    img = img.unsqueeze(0)
    label = self.train_dataset.targets[idx]
    return {"img":img,"label":label}
  
  def __len__(self):
    return 1000


In [ ]:
ds = dataset()

In [ ]:
#porting dataset to hf dataset
!pip install datasets
from datasets import Dataset

In [ ]:
def gen():
  for idx in ds:
    yield {"img":idx["img"],"label":idx["label"]}

In [ ]:
ds = Dataset.from_generator(gen)

In [ ]:
ds = ds.train_test_split(test_size=0.2)

In [ ]:
def preprocess_image(examples):
  images = examples["img"]
  images = [np.array(image, dtype=np.uint8) for image in images]
  return examples

ds = ds.map(preprocess_image,batched=True)
ds.set_format(type="torch")

In [ ]:
from transformers import DefaultDataCollator

class datacollator(DefaultDataCollator):
  def __call__(self,features,return_tensors=None):
    batch = {}
    batch['imgs'] = features["img"]
    batch['labels'] = features["labels"]
    return batch

datacollator = DefaultDataCollator()

In [ ]:
#####porting model#######
#!pip install transformers

In [ ]:
TFout = transformers.modeling_outputs.SequenceClassifierOutput

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((14, 14))
        self.fc1 = nn.Linear(3136, 10)  

    def forward(self, x, labels=None):
        x = self.conv1(x.float())
        x = self.relu(x)
        x = self.pool(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc1(x)
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
        
        return TFout(
            loss=loss,
            logits=logits,
            hidden_states=None,
            attentions=None
        )

model = CNN()


In [ ]:
args = TrainingArguments(
    "out_dir",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="wandb"
)

PyTorch: setting up devices


In [ ]:
ct = CTrainer
dl = torch.utils.data.DataLoader(ds["train"],batch_size=2,shuffle=True)


In [ ]:
ds["train"]

Dataset({
    features: ['img', 'label'],
    num_rows: 800
})

In [ ]:
#dc = datacollator(ds["train"],return_tensors="pt")##same things as collate_function in pytorch

In [ ]:
#dc["img"].shape

In [ ]:
# Assuming you have defined your DataLoader as `data_loader`
for batch in dl:
    #g = batch["img"]
    #logits = model(g, batch["labels"])
    print(batch["img"].shape)
    break


torch.Size([2, 1, 28, 28])


In [ ]:
for i in dl:
  #g = torch.stack([im.get("img") for im in ])
  #print(g)
  print(model(i["img"],i["label"]).loss)
  

In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
# wandb.init(
#     project="sHF",
#     name="shittyhf",
# )

In [ ]:
class CTrainer(Trainer):

  def get_train_dataloader(self):
    return torch.utils.data.DataLoader(ds["train"],batch_size=4,shuffle=True)
  
  def get_eval_dataloader(self):
    return torch.utils.data.DataLoader(ds["test"],batch_size=4,shuffle=True)

  def compute_loss(self, model, inputs, return_outputs=False):
    loss = model(i["img"],i["label"]).loss
    logits = model(i["img"],i["label"]).logits
    return (loss, outputs) if return_outputs else loss


  
trainer = CTrainer(
    model=model,
    args=args,
)

In [ ]:
trainer.train()
wandb.finish()

***** Running training *****
  Num examples = 800
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 600
  Number of trainable parameters = 31,530
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
500,0.937600


Saving model checkpoint to out_dir/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




real_loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁█
train/global_step,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
train/learning_rate,▁
train/loss,▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
real_loss,0.0
